In [2]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
%matplotlib inline

df_test = pd.read_csv('cs5228-2020-semester-2-final-project/test.csv')
df_init = df_test.copy()
df_init['remaining_lease'] = df_init['lease_commence_date'].astype(int) - 1918
df_init['year'] = df_init['month'].apply(lambda x: int(x[:4]))

df_init['street_name'] = df_init['street_name'].str.replace('UPP ', 'UPPER ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' CTRL', ' CENTRAL', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('JLN ', 'JALAN ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('BT ', 'BUKIT ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('LOR ', 'LORONG ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' NTH', ' NORTH', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('NTH BRIDGE', 'NORTH BRIDGE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('KG', 'KAMPONG', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' DR', ' DRIVE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('TG PAGAR', 'TANJONG PAGAR', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("C'WEALTH", 'COMMONWEALTH', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("WOODLANDS DRIVE 75", 'WOODLAND DRIVE 75', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("TAMAN HO SWEE", 'JALAN BUKIT HO SWEE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("YUNG PING RD", '157 CORPORATION DRIVE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("YUNG LOH RD", '157 CORPORATION DRIVE', regex = True)
address_set = df_init["street_name"].unique().tolist()

df_primary_schools = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-primary-schools.csv')
df_secondary_schools = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-secondary-schools.csv')
df_commerical = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-commerical-centres.csv')
df_hawker = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-gov-markets-hawker-centres.csv')
df_malls = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-shopping-malls.csv')
df_stations = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-train-stations.csv')
df_demographics = pd.read_csv('cs5228-2020-semester-2-final-project/auxiliary-data/sg-population-demographics.csv')

In [4]:
import geopy
import geopy.distance
# df_init["lat_long"] = df_init[['latitude', 'longitude']].apply(geopy.Point, axis=1)
# df_unique_point = df_init.groupby(['latitude', 'longitude']).size().reset_index(name='Freq')
# df_unique_point["lat_long"] = df_unique_point[['latitude', 'longitude']].apply(geopy.Point, axis=1)
# df_unique_point

# def calculate_nearest(venue_name, venue_df):
#     df_unique_point["lat_long"] = df_unique_point[['latitude', 'longitude']].apply(geopy.Point, axis=1)
#     venue_distance = pd.DataFrame(df_unique_point['lat_long'])
#     venue_df["lat_long"] = venue_df[['lat', 'lng']].apply(geopy.Point, axis=1)
#     venue_list = list(venue_df.T.iloc[-1,:])
#     for x in venue_list:
#         venue_distance[tuple(x)] = df_unique_point['lat_long'].apply(lambda y: geopy.distance.geodesic(y,x).km)
#     venue_distance["lat_long"] = venue_distance["lat_long"].apply(tuple)
#     feature_name = str('distance_to_nearest_'+venue_name)
#     venue_distance[feature_name] = venue_distance.iloc[:,1:].apply(lambda x: min(x), axis=1)
#     venue_distance = venue_distance.iloc[:,[0,-1]]
#     df_unique_point["lat_long"] = df_unique_point["lat_long"].apply(tuple)
#     df_with_venues = pd.merge(df_unique_point, venue_distance, on='lat_long', how='left')
#     return df_with_venues

# df_primary_schools = df_primary_schools.dropna()
# df_secondary_schools = df_secondary_schools.dropna()
# df_commerical = df_commerical.dropna()
# df_hawker = df_hawker.dropna()
# df_malls = df_malls.dropna()
# df_stations = df_stations.dropna()
# df_demographics = df_demographics.dropna()

# df_primary_schools = calculate_nearest('primary_sch',df_primary_schools)
# df_secondary_schools = calculate_nearest('second_sch',df_secondary_schools)
# df_commercial = calculate_nearest('commercial',df_commerical)
# df_hawker = calculate_nearest('hawker',df_hawker)
# df_malls = calculate_nearest('malls',df_malls)
# df_stations = calculate_nearest('stations',df_stations)

# df_init['lat_long'] =  df_init["lat_long"].apply(tuple)
# df_merged_venue = pd.merge(df_init, df_stations, on='lat_long', how='left')
# df_merged_venue = pd.merge(df_merged_venue, df_primary_schools, on='lat_long', how='left')
# df_merged_venue = pd.merge(df_merged_venue, df_secondary_schools, on='lat_long', how='left')
# df_merged_venue = pd.merge(df_merged_venue, df_commercial, on='lat_long', how='left')
# df_merged_venue = pd.merge(df_merged_venue, df_hawker, on='lat_long', how='left')
# df_merged_venue = pd.merge(df_merged_venue, df_malls, on='lat_long', how='left')
# df_merged_venue

df_merged_venue = df_merged_venue[['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'eco_category', 'lease_commence_date',
       'elevation', 'subzone', 'planning_area',
       'region', 'remaining_lease', 
       'distance_to_nearest_stations',
       'distance_to_nearest_primary_sch', 
       'distance_to_nearest_second_sch', 
       'distance_to_nearest_commercial', 
       'distance_to_nearest_hawker',
       'distance_to_nearest_malls']]

In [23]:
print(df_merged_venue['flat_model'].nunique())
df_merged_venue['flat_model'] = df_merged_venue['flat_model'].str.replace('2-ROOM', '2-room') \
                                    .str.replace('MULTI GENERATION', 'Multi Generation') \
                                    .str.replace('PREMIUM APARTMENT', 'Premium Apartment') \
                                    .str.replace('Premium Apartment.', 'Premium Apartment') \
                                    .str.replace('Premium Apartment Loft', 'Premium Apartment') \
                                    .str.replace('NEW GENERATION', 'New Generation') \
                                    .str.replace('MODEL A', 'Model A') \
                                    .str.replace('Improved-MAISONETTE', 'Maisonette') \
                                    .str.replace('Model A-Maisonette', 'Maisonette') \
                                    .str.replace('Premium ApartmentLoft', 'Premium Apartment') \
                                    .str.replace('flat_type_MULTI-GENERATION', 'flat_type_MULTI GENERATION') \
                                    .str.replace('flat_model_PREMIUM Apartment', 'flat_model_Premium Apartment') \
                                    .str.replace('Model A-MAISONETTE', 'Maisonette') \
                                    .str.replace('APARTMENT', 'Apartment') \
                                    .str.replace('MAISONETTE', 'Maisonette') \
                                    .str.replace('IMPROVED', 'Improved') \
                                    .str.replace('SIMPLIFIED', 'Simplified') \
                                    .str.replace('MODEL A-MAISONETTE', 'Maisonette') \
                                    .str.replace('Improved-Maisonette', 'Maisonette') \
                                    .str.replace('Premium Maisonette', 'Maisonette') \
                                    .str.replace('ADJOINED FLAT', 'Adjoined flat') \
                                    .str.replace('TERRACE', 'Terrace') 

df_merged_venue['flat_model'].nunique()
df_merged_venue['flat_type'].unique()
df_merged_venue['flat_type'] = df_merged_venue['flat_type'].str.replace('5-room', '5 room')\
                                                           .str.replace('4-room', '4 room')\
                                                           .str.replace('3-room', '3 room')\
                                                           .str.replace('2-room', '2 room')\
                                                           .str.replace('1-room', '1 room')
df_merged_venue['flat_type'].unique()

df_merged_venue['toilet_number'] = df_merged_venue['flat_type'].map({
    '1 room': 1,
    '2 room': 1,
    '3 room': 2,
    '4 room': 2,
    '5 room': 2,
    'executive': 2,
    'multi generation': 3
})
df_merged_venue['toilet_number'] = df_merged_venue['toilet_number'].astype(int)

df_merged_venue['room_number'] = df_merged_venue['flat_type'].map({
    '1 room': 1,
    '2 room': 2,
    '3 room': 3,
    '4 room': 4,
    '5 room': 5,
    'executive': 3,
    'multi generation': 4
})
df_merged_venue['room_number'] = df_merged_venue['room_number'].astype(int)

df_onehot = pd.get_dummies(df_merged_venue[['flat_type', 'flat_model', 'storey_range', 'town']])
df_main_onehot = df_merged_venue.join(df_onehot)      
df_main_onehot.drop(['town', 'eco_category', 'subzone', \
               'block','lease_commence_date','subzone', 'region', 'elevation'], 
              axis = 1, inplace = True)
df_main_onehot['storey_avg'] = df_main_onehot['storey_range'].apply(lambda x: int(x[-2:]))
   
df_main_onehot['storey < 10'] = df_main_onehot['storey_range_01 to 03']\
                        + df_main_onehot['storey_range_01 to 05']\
                        + df_main_onehot['storey_range_04 to 06']\
                        + df_main_onehot['storey_range_06 to 10']\
                        + df_main_onehot['storey_range_07 to 09']\
                
df_main_onehot['storey < 15'] = df_main_onehot['storey_range_10 to 12']\
                        + df_main_onehot['storey_range_11 to 15'] 
                                   
df_main_onehot['storey < 20'] = df_main_onehot['storey_range_16 to 18']\
                        + df_main_onehot['storey_range_16 to 20'] 
    
df_main_onehot['storey < 30'] = df_main_onehot['storey_range_19 to 21']\
                        + df_main_onehot['storey_range_21 to 25']\
                        + df_main_onehot['storey_range_22 to 24']\
                        + df_main_onehot['storey_range_06 to 10']\
                        + df_main_onehot['storey_range_25 to 27']\
                        + df_main_onehot['storey_range_26 to 30']\
                        + df_main_onehot['storey_range_28 to 30']
                        
df_main_onehot['story > 30'] = df_main_onehot['storey_range_31 to 33']\
                        + df_main_onehot['storey_range_34 to 36']\
                        + df_main_onehot['storey_range_36 to 40']\
                        + df_main_onehot['storey_range_37 to 39']\
                        + df_main_onehot['storey_range_40 to 42']\
                        + df_main_onehot['storey_range_43 to 45']\
                        + df_main_onehot['storey_range_46 to 48']\
                        + df_main_onehot['storey_range_49 to 51']
                        
df_main_onehot.drop(['storey_range_01 to 03',
                'storey_range_01 to 05',
                'storey_range_04 to 06',
                'storey_range_06 to 10',
                'storey_range_07 to 09',
                'storey_range_10 to 12',
                'storey_range_11 to 15',
                'storey_range_13 to 15',
                'storey_range_16 to 18',
                'storey_range_16 to 20',
                'storey_range_19 to 21',
                'storey_range_21 to 25',
                'storey_range_22 to 24',
                'storey_range_25 to 27',
                'storey_range_26 to 30',
                'storey_range_28 to 30',
                'storey_range_31 to 33',
                'storey_range_34 to 36',
                'storey_range_36 to 40',
                'storey_range_37 to 39',
                'storey_range_40 to 42',
                'storey_range_43 to 45',
                'storey_range_46 to 48',
                'storey_range_49 to 51','storey_range'], 
                axis = 1, inplace = True)
df_main_onehot['year'] = df_main_onehot['month'].apply(lambda x: int(x[:4]))

20


<ipython-input-23-2144621d5e54>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged_venue['flat_model'] = df_merged_venue['flat_model'].str.replace('2-ROOM', '2-room') \
<ipython-input-23-2144621d5e54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_venue['flat_model'] = df_merged_venue['flat_model'].str.replace('2-ROOM', '2-room') \
<ipython-input-23-2144621d5e54>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_venue['flat_type

In [24]:
df_main_onehot

,month,flat_type,street_name,floor_area_sqm,flat_model,planning_area,remaining_lease,distance_to_nearest_stations,distance_to_nearest_primary_sch,distance_to_nearest_second_sch,...,town_toa payoh,town_woodlands,town_yishun,storey_avg,storey < 10,storey < 15,storey < 20,storey < 30,story > 30,year
0,2004-01,4 room,bukit batok west avenue 6,94.0,new generation,bukit batok,71,0.667554,0.466245,0.426156,...,0,0,0,6,1,0,0,0,0,2004
1,2001-11,5 room,tampines street 34,122.0,improved,tampines,79,0.767025,0.527107,0.078546,...,0,0,0,6,1,0,0,0,0,2001
2,2002-07,3 room,jurong east street 21,67.0,new generation,jurong east,64,0.515380,0.574280,0.400034,...,0,0,0,3,1,0,0,0,0,2002
3,2015-04,3 room,Ang Mo Kio Avenue 5,82.0,new generation,ang mo kio,63,0.547114,0.799705,0.568718,...,0,0,0,6,1,0,0,0,0,2015
4,2004-04,5 room,clementi avenue 2,117.0,standard,clementi,60,0.516298,0.316165,0.443550,...,0,0,0,3,1,0,0,0,0,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107929,2008-04,5 room,buangkok crescent,110.0,improved,hougang,85,1.557685,0.301777,0.460324,...,0,0,0,12,0,1,0,0,0,2008
107930,2006-01,4 room,upper boon keng road,102.0,model a,kallang,81,0.346682,1.050435,1.465030,...,0,0,0,15,0,0,0,0,0,2006
107931,2000-01,3 room,beach road,68.0,improved,downtown core,61,0.221181,1.040489,0.746349,...,0,0,0,9,1,0,0,0,0,2000
107932,2009-07,4 room,jurong west street 91,104.0,model a,jurong west,70,1.138997,0.100565,0.184384,...,0,0,0,12,0,1,0,0,0,2009


In [25]:
len(df_main_onehot.columns),df_main_onehot.columns

(75,
 Index(['month', 'flat_type', 'street_name', 'floor_area_sqm', 'flat_model',
        'planning_area', 'remaining_lease', 'distance_to_nearest_stations',
        'distance_to_nearest_primary_sch', 'distance_to_nearest_second_sch',
        'distance_to_nearest_commercial', 'distance_to_nearest_hawker',
        'distance_to_nearest_malls', 'toilet_number', 'room_number',
        'flat_type_1 room', 'flat_type_2 room', 'flat_type_3 room',
        'flat_type_4 room', 'flat_type_5 room', 'flat_type_executive',
        'flat_type_multi generation', 'flat_model_2 room',
        'flat_model_adjoined flat', 'flat_model_apartment', 'flat_model_dbss',
        'flat_model_improved', 'flat_model_improved maisonette',
        'flat_model_maisonette', 'flat_model_model a',
        'flat_model_model a maisonette', 'flat_model_model a2',
        'flat_model_multi generation', 'flat_model_new generation',
        'flat_model_premium apartment', 'flat_model_premium apartment loft',
        'flat_mode

In [26]:
df_main_onehot = df_main_onehot.drop(columns=['flat_type','street_name',
       'flat_model', 'planning_area'])

In [27]:
df_main_onehot.to_csv('test_onehot.csv')

In [ ]:
scaler = StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)

yhat = lasso.predict(X_trains)